# Neo4j GraphRAG test

https://github.com/neo4j-product-examples/graphrag-python-examples/blob/main/end-to-end-lupus.ipynb

In [3]:
from dotenv import load_dotenv
import os

# load neo4j credentials (and openai api key in background).
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

#uncomment this line if you aren't using a .env file
# os.environ['OPENAI_API_KEY'] = 'copy_paste_the_openai_key_here'

In [4]:
import openai
import neo4j
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.llm import AzureOpenAILLM
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('.env')

# from neo4j_graphrag.llm import OpenAILLM
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
# ex_llm=OpenAILLM(
#     model_name=os.getenv("GRAPHRAG_LLM_MODEL"),
#     model_params={
#         "response_format": {"type": "json_object"}, # use json_object formatting for best results
#         "temperature": 0 # turning temperature down for more deterministic results
#     }
# )

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

ex_llm = AzureOpenAILLM(
    model_name=os.getenv("GRAPHRAG_LLM_MODEL"),
    azure_endpoint=os.getenv("GRAPHRAG_API_BASE"),  # update with your endpoint
    api_version=os.getenv("GRAPHRAG_API_VERSION"),  # update appropriate version
    api_key=os.getenv("GRAPHRAG_API_KEY"),  # api_key is optional and can also be set with OPENAI_API_KEY env var
)

#create text embedder
embedder = OpenAIEmbeddings()

In [5]:
#define node labels
basic_node_labels = ["Object", "Entity", "Group", "Person", "Organization", "Place"]

academic_node_labels = ["ArticleOrPaper", "PublicationOrJournal"]

medical_node_labels = ["Anatomy", "BiologicalProcess", "Cell", "CellularComponent", 
                       "CellType", "Condition", "Disease", "Drug",
                       "EffectOrPhenotype", "Exposure", "GeneOrProtein", "Molecule",
                       "MolecularFunction", "Pathway"]

node_labels = basic_node_labels + academic_node_labels + medical_node_labels

# define relationship types
rel_types = ["ACTIVATES", "AFFECTS", "ASSESSES", "ASSOCIATED_WITH", "AUTHORED",
    "BIOMARKER_FOR", "CAUSES", "CITES", "CONTRIBUTES_TO", "DESCRIBES", "EXPRESSES",
    "HAS_REACTION", "HAS_SYMPTOM", "INCLUDES", "INTERACTS_WITH", "PRESCRIBED",
    "PRODUCES", "RECEIVED", "RESULTS_IN", "TREATS", "USED_FOR"]

In [6]:
prompt_template = '''
You are a medical researcher tasks with extracting information from papers 
and structuring it in a property graph to inform further medical and research Q&A.

Extract the entities (nodes) and specify their type from the following Input text.
Also extract the relationships between these nodes. the relationship direction goes from the start node to the end node. 


Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "Description of the relationship"}} }}] }}

- Use only the information from the Input text.  Do not add any additional information.  
- If the input text is empty, return empty Json. 
- Make sure to create as many nodes and relationships as needed to offer rich medical context for further research.
- An AI knowledge assistant must be able to read this graph and immediately understand the context to inform detailed research questions. 
- Multiple documents will be ingested from different sources and we are using this property graph to connect information, so make sure entity types are fairly general. 

Use only fhe following nodes and relationships (if provided):
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Do not return any additional information other than the JSON in it.

Examples:
{examples}

Input text:

{text}
'''

In [7]:
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder_pdf = SimpleKGPipeline(
    llm=ex_llm,
    driver=driver,
    text_splitter=FixedSizeSplitter(chunk_size=500, chunk_overlap=100),
    embedder=embedder,
    entities=node_labels,
    relations=rel_types,
    prompt_template=prompt_template,
    from_pdf=True
)

pdf_file_paths = ['/Users/bingranyou/Documents/GitHub_Mac_mini/DeepTutor/input_files/RankRAG- Unifying Context Ranking with  Retrieval-Augmented Generation in LLMs.pdf']

from neo4j import GraphDatabase

def document_exists(driver, file_path):
    query = """
    MATCH (d:Document {path: $file_path})
    RETURN d
    """
    with driver.session() as session:
        result = session.run(query, file_path=file_path)
        return result.single() is not None

for path in pdf_file_paths:
    print(f"Processing: {path}")
    if document_exists(driver, path):
        print(f"Skipping {path}; already processed.")
        continue
    else:
        print(f"Processing : {path}")
        pdf_result = await kg_builder_pdf.run_async(file_path=path)
        print(f"Result: {pdf_result}")
    pdf_result = await kg_builder_pdf.run_async(file_path=path)
    print(f"Result: {pdf_result}")

Processing: /Users/bingranyou/Documents/GitHub_Mac_mini/DeepTutor/input_files/RankRAG- Unifying Context Ranking with  Retrieval-Augmented Generation in LLMs.pdf
Skipping /Users/bingranyou/Documents/GitHub_Mac_mini/DeepTutor/input_files/RankRAG- Unifying Context Ranking with  Retrieval-Augmented Generation in LLMs.pdf; already processed.


/Users/bingran_you/.local/lib/python3.12/site-packages/neo4j/_sync/work/result.py:620: UserWarning: Expected a result with a single record, but found multiple.
  warn(


## Knowledge Graph Retrieval
We will leverage Neo4j's vector search capabilities here. To do this, we need to begin by creating a vector index on the text chunks from the PDFs, which are stored on Chunk nodes in our knowledge graph.

In [8]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                    embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

In [9]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

In [10]:
import json

vector_res = vector_retriever.get_search_results(query_text = "What is RankRAG?", 
                                                 top_k=5)
for i in vector_res.records: print("====\n" + json.dumps(i.data(), indent=4))

====
{
    "node": {
        "text": "n the evaluations of ranking associated with the RAG\ntasks, even surpassing the LLMs fine-tuned with 10\u00d7more ranking data. We attribute this success\nto the transferable design of RankRAG training.\n\u2022We extensively compare the proposed RankRAG method with several strong baselines, including\nthe open-sourced ChatQA-1.5. On nine general-domain and five biomedical knowledge-intensive\nbenchmarks for RAG, Llama3-RankRAG-8B and Llama3-RankRAG-70B outperforms Llama3-\nChatQA-1.5-8B and Llama3-ChatQA-1.5-7"
    },
    "nodeLabels": [
        "__KGBuilder__",
        "Chunk"
    ],
    "elementId": "4:cbfd137b-822c-461a-a486-5bb936f239f6:1585",
    "id": "4:cbfd137b-822c-461a-a486-5bb936f239f6:1585",
    "score": 0.926300048828125
}
====
{
    "node": {
        "text": "n the evaluations of ranking associated with the RAG\ntasks, even surpassing the LLMs fine-tuned with 10\u00d7more ranking data. We attribute this success\nto the transferable d

In [11]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, 
  collect(DISTINCT rel) AS rels

//3) format and return context
RETURN '=== text ===\n' + apoc.text.join([c in chunks | c.text], '\n---\n') + '\n\n=== kg_rels ===\n' +
  apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], '\n---\n') AS info
"""
)

In [12]:
vc_res = vc_retriever.get_search_results(query_text = "What is multiplexing", top_k=3)

# print output
kg_rel_pos = vc_res.records[0]['info'].find('\n\n=== kg_rels ===\n')
print("# Text Chunk Context:")
print(vc_res.records[0]['info'][:kg_rel_pos])
print("# KG Context From Relationships:")
print(vc_res.records[0]['info'][kg_rel_pos:])

# Text Chunk Context:
=== text ===
6]. Ontheotherhand,
implementing a multiplexed light-matter interface with
these systems is technically challenging. Towards over-
coming this problem, a few multiplexing schemes have
already been proposed for ion and atom-based quan-
tum processors [27–30]. The only reported experimental
work, we are aware of, is the demonstration of multiplex-
ing using a static three-ion chain [15]. In view of the re-
cent advances of the quantum CCD architecture [31–33],
a complementary approach to multiplex
---
6]. Ontheotherhand,
implementing a multiplexed light-matter interface with
these systems is technically challenging. Towards over-
coming this problem, a few multiplexing schemes have
already been proposed for ion and atom-based quan-
tum processors [27–30]. The only reported experimental
work, we are aware of, is the demonstration of multiplex-
ing using a static three-ion chain [15]. In view of the re-
cent advances of the quantum CCD architecture [31–33

## GraphRAG

In [13]:
import openai
import neo4j
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('.env')

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

ex_llm = AzureOpenAILLM(
    model_name=os.getenv("GRAPHRAG_LLM_MODEL"),
    azure_endpoint=os.getenv("GRAPHRAG_API_BASE"),  # update with your endpoint
    api_version=os.getenv("GRAPHRAG_API_VERSION"),  # update appropriate version
    api_key=os.getenv("GRAPHRAG_API_KEY"),  # api_key is optional and can also be set with OPENAI_API_KEY env var
)

#create text embedder
embedder = OpenAIEmbeddings()

In [ ]:
from neo4j_graphrag.llm import AzureOpenAILLM
from neo4j_graphrag.generation import RagTemplate
from neo4j_graphrag.generation.graphrag import GraphRAG

llm = AzureOpenAILLM(
    model_name=os.getenv("GRAPHRAG_LLM_MODEL"),
    azure_endpoint=os.getenv("GRAPHRAG_API_BASE"),  # update with your endpoint
    api_version=os.getenv("GRAPHRAG_API_VERSION"),  # update appropriate version
    api_key=os.getenv("GRAPHRAG_API_KEY"),  # api_key is optional and can also be set with OPENAI_API_KEY env var
)

rag_template = RagTemplate(template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned. 

# Question:
{query_text}

# Context:
{context}

# Answer:
''', expected_inputs=['query_text', 'context'])

v_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)
vc_rag = GraphRAG(llm=llm, retriever=vc_retriever, prompt_template=rag_template)

In [16]:
q = "What is multiplexing."
print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k':5}).answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k':5}).answer}")

Vector Response: 
Multiplexing is the process of ion-transport through a specific spatial location with maximized photon coupling efficiency.


Vector + Cypher Response: 
Multiplexing is a scheme to implement a light-matter interface, technically challenging with certain systems, and in quantum processors, it involves schemes for ion and atom-based systems. A complementary approach to multiplexing mentioned is ion-transport through a specific spatial location with maximized photon coupling efficiency.


In [17]:
q = "What is quatnum network"

v_rag_result = v_rag.search(q, retriever_config={'top_k': 5}, return_context=True)
vc_rag_result = vc_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

print(f"Vector Response: \n{v_rag_result.answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag_result.answer}")

Vector Response: 
A quantum network involves the entanglement between matter qubits and photonic qubits. This entanglement can be distributed over long distances, such as a 101-km-long fiber channel. The network can enhance processing capabilities, for example, by using multiple cotrapped matter qubits at a node.


Vector + Cypher Response: 
A key requirement for long-distance quantum networking is the ability to entangle a matter qubit with a photon and to distribute that photon over many tens of kilometers.


In [18]:
for i in v_rag_result.retriever_result.items: print(json.dumps(eval(i.content), indent=1))

{
 "text": "ned for the most advanced forms of a quantum\nnetwork [ 2].\nIn this paper, we present two main results. First, entan-\nglement between a matter qubit and a photonic qubit is\nachieved over a spooled 101-km-long \ufb01ber channel: twice\nthe distance of previous works (see, e.g., Refs. [ 15\u201318])\nand requiring a matter-qubit coherence time on the order of\nthe photon travel time (494 \u00b5s) to achieve. Second, using\nthree cotrapped matter qubits in the node, we demonstrate\na multimoding enhancement for the rat"
}
{
 "text": "ned for the most advanced forms of a quantum\nnetwork [ 2].\nIn this paper, we present two main results. First, entan-\nglement between a matter qubit and a photonic qubit is\nachieved over a spooled 101-km-long \ufb01ber channel: twice\nthe distance of previous works (see, e.g., Refs. [ 15\u201318])\nand requiring a matter-qubit coherence time on the order of\nthe photon travel time (494 \u00b5s) to achieve. Second, using\nthree cotrapped matt

In [19]:
vc_ls = vc_rag_result.retriever_result.items[0].content.split('\\n---\\n')
for i in vc_ls:
    if "biomarker" in i: print(i)

In [20]:
vc_ls = vc_rag_result.retriever_result.items[0].content.split('\\n---\\n')
for i in vc_ls:
    if "treat" in i: print(i)

In [21]:
q = "What is RankRAG?"
print(f"Vector Response: \n{v_rag.search(q, retriever_config={'top_k': 5}).answer}")
print("\n===========================\n")
print(f"Vector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k': 5}).answer}")

Vector Response: 
RankRAG is a method designed for retrieval-augmented generation (RAG) tasks in NLP. It has been evaluated against several strong baselines and demonstrated superior performance, even surpassing models fine-tuned with significantly more ranking data. This success is attributed to the transferable design of RankRAG training. RankRAG models, such as Llama3-RankRAG-8B and Llama3-RankRAG-70B, outperform other models like Llama3-ChatQA-1.5-8B and Llama3-ChatQA-1.5-70B across various benchmarks.


Vector + Cypher Response: 
RankRAG is a method used for answer generation and context ranking in the evaluations of ranking associated with the RAG (Retrieval-Augmented Generation) tasks.
